<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-3/DL_Assignment_3_with_attention_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Activation
from keras.layers import Concatenate,TimeDistributed
import torch
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU,SimpleRNN
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN,GRU
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
import math
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU,SimpleRNN
from keras.layers import Dropout
import random
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train="/content/gdrive/MyDrive/hi.translit.sampled.train.tsv"
test="/content/gdrive/MyDrive/hi.translit.sampled.test.tsv"
val="/content/gdrive/MyDrive/hi.translit.sampled.dev.tsv"


In [ ]:

# Vectorize the data.
input_texts = []
target_texts = []

with open(train, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")

    ##Appending '\t' and '\n' for every word in target_texts
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)




In [ ]:
"""Preparing set for unique imput and target characters"""
input_characters = set()
target_characters = set()

for i in range( len(input_texts)):
  for char in input_texts[i]:
    input_characters.add(char)

  for char in target_texts[i]:
    target_characters.add(char)


#Convert set to list
input_characters = list(input_characters)
target_characters = list(target_characters)

In [ ]:
"""Preparing dictionary for input characters and target characters"""
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

##Finding the length of maximum input word and maximum target word

max_encoder_seq_length=-1
for i in input_texts:
  if len(i)>max_encoder_seq_length:
    max_encoder_seq_length=len(i)

max_decoder_seq_length=-1
for i in target_texts:
  if len(i)>max_decoder_seq_length:
    max_decoder_seq_length=len(i)

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

target_token_index[" "]=65
input_token_index[" "]=26

In [ ]:
max_encoder_seq_length, max_decoder_seq_length

(20, 21)

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype="float32"
)

In [ ]:
# For validation data
input_texts_val = []
target_texts_val = []

with open(val, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text=input_text
    target_text = "\t" + target_text + "\n"
    input_texts_val.append(input_text)
    target_texts_val.append(target_text)


encoder_input_data_val = np.zeros(
    (len(input_texts_val), max_encoder_seq_length), dtype="float32"
)


for i, input_text in enumerate(input_texts_val):
    for t, char in enumerate(input_text):
        encoder_input_data_val[i, t] =  input_token_index[char]

In [ ]:
##For test data
# For validation data
input_texts_test = []
target_texts_test = []

with open(test, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text=input_text
    target_text = "\t" + target_text + "\n"
    input_texts_test.append(input_text)
    target_texts_test.append(target_text)


encoder_input_data_test = np.zeros(
    (len(input_texts_test), max_encoder_seq_length), dtype="float32"
)


for i, input_text in enumerate(input_texts_test):
    for t, char in enumerate(input_text):
        encoder_input_data_test[i, t] =  input_token_index[char]

In [ ]:
##For test data
input_texts_test = []
target_texts_test = []

with open(test, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text=input_text
    target_text = "\t" + target_text + "\n"
    input_texts_test.append(input_text)
    target_texts_test.append(target_text)


for i, input_text in enumerate(input_texts_test):
    for t, char in enumerate(input_text):
        encoder_input_data_test[i, t] =  input_token_index[char]

In [ ]:
reverse_input_char_index=dict()
for char, i in input_token_index.items():
  reverse_input_char_index[i]=char

reverse_target_char_index=dict()
for char, i in target_token_index.items():
  reverse_target_char_index[i]=char

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN
from keras.layers import dot
from tensorflow.keras.layers import Activation
from keras.layers import concatenate
from keras.layers import Attention,TimeDistributed

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('attention.py','wb').write(src)

Saving attention.py to attention.py


4645

In [ ]:
from attention import AttentionLayer

In [ ]:

#### Testing of accuracy word by word check


def ValAcc2(x, y, enc_model,dec_model, beam_size):
  #onehotlist=[]
  correct_predict=0
  pred_words=[]
  temp=""
  for i, input_text in enumerate(x):

    input_seq = x[i:i+1]
    decoded_word = decode_sequence1(input_seq, enc_model, dec_model,beam_size)

    temp=decoded_word
    #print(decoded_word)
    #print("---",y[i])
    pred_words.append(decoded_word+'\n')

    true_word=y[i].split('\t')[1].split('\n')[0]
    if decoded_word==true_word:
      correct_predict=correct_predict+1

    #print("Decoded_word: ", decoded_word, "\tTrue_word: ",true_word)
  
  
  valid_acc=correct_predict/len(x)    
  return valid_acc, pred_words  #, decoded_word #,correct_predict,temp






In [ ]:
def beam_search_decoder(predictions, target_sequence_index,top_k=3):

  #initially
  output_sequences=[("",0)]

  for token_probs in predictions:
    new_sequences=[]

    #append new tokens to old sequences and re-score
    for old_seq, old_score in output_sequences:
      for char_index in range(len(token_probs)):
        new_seq = old_seq+target_sequence_index[char_index] #Changed from original code

        #considering log-likelihood for scoring
        new_score= old_score + math.log(token_probs[char_index]) #Changed from original code

        #DOUBT: logging based of ASCII Value might be incorrect --> Clear

        new_sequences.append((new_seq, new_score))

    
    output_sequences=sorted(new_sequences, key= lambda val: val[1], reverse=True)

    #To select top-k based on score
    output_sequences = output_sequences[:top_k]

  
  return output_sequences